<a href="https://colab.research.google.com/github/Marysia-max/SRMiO/blob/main/Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics roboflow opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 64.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
from ultralytics import YOLO
from roboflow import Roboflow


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="LbYkfCVpxzkX61r7Alu5")
project = rf.workspace("gentagabriele").project("clothing-olr9p")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:

print("Sciezka do danych :", dataset.location)


In [ ]:

!find {dataset.location} -maxdepth 3 -type f -name "data.yaml" -print


In [ ]:
import yaml, pathlib
DATA_YAML = "/content/clothing-1/data.yaml"
print("Sciezka:", DATA_YAML)

with open(DATA_YAML, "r") as f:
    y = yaml.safe_load(f)
print("Klasy:", y["names"])


In [ ]:
!pip -q install "ultralytics>=8,<9"
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results = model.train(
    data=DATA_YAML,
    epochs=60,
    imgsz=640,
    batch=16,
    device=0,
    patience=20,
    cos_lr=True,
    augment=True,
    project="runs_clothes",
    name="yolov8n_clothes"
)


In [ ]:
!pip install ultralytics gradio


In [ ]:
from ultralytics import YOLO


model = YOLO("/content/runs_clothes/yolov8n_clothes/weights/best.pt")
print(model.names)

In [ ]:
import numpy as np

def predict(image):
    # image przychodzi jako tablica numpy (H, W, 3)
    results = model(image)[0]          # bierzemy pierwszy wynik
    boxes = results.boxes
    names = model.names

    labels = []
    for box in boxes:
        cls_id = int(box.cls[0].item())
        labels.append(names[cls_id])

    # rysowanie bounding boxów i etykiet na obrazie:
    annotated = results.plot()         # zwraca obraz z narysowanymi boxami

    # unikalne nazwy klas
    unique_labels = ", ".join(sorted(set(labels))) if labels else "Nic nie wykryto"

    return annotated, unique_labels


In [ ]:
import gradio as gr

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(
        sources=["upload", "webcam"],  # tutaj włączasz opcję kamery
        type="numpy",
        streaming=False
    ),
    outputs=[
        gr.Image(label="Detekcje"),
        gr.Textbox(label="Rozpoznane ubrania")
    ],
    title="Rozpoznawanie ubrań YOLO"
)

demo.launch(share=True)



In [5]:
import os

# zmień na ścieżkę do datasetu
path = "/content/clothing-1"   # przykład

train = os.path.join(path, "train/images")
valid = os.path.join(path, "valid/images")
test  = os.path.join(path, "test/images")

print("Liczba obrazów w train:", len(os.listdir(train)))
print("Liczba obrazów w valid:", len(os.listdir(valid)))
print("Liczba obrazów w test:", len(os.listdir(test)))


FileNotFoundError: [Errno 2] No such file or directory: '/content/clothing-1/train/images'